In [ ]:
import sys

from SALib.analyze import delta
from SALib.util import read_param_file
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from numpy import random
from SALib.analyze import rbd_fast

In [ ]:
df = pd.read_csv("summaries_round_3.csv", sep=';')

df['ElecRate'] = 0.42
df["Pop2030"] = 15672000

df['ElecRate'] = np.where(df['Country'] == 'na', 0.54, df['ElecRate'])
df['ElecRate'] = np.where(df['Country'] == 'mw', 0.18, df['ElecRate'])

df['Pop2030'] = np.where(df['Country'] == 'na', 3010873, df['Pop2030'] )
df['Pop2030'] = np.where(df['Country'] == 'mw', 19130000, df['Pop2030'])

cols = ['EDemand',
 'GridGenCost',
 'PVCost',
 'GridCapInvCost',
 'DiscountRate',
 'LVCost',
 'MVCost',
 'GridLosses',
 'Core',
 'Buffer',
 'Admin',
 'Method',
 'Res',
 'ElecRate',
 'NatPopDens',
 'Pop2030',
 'PopGrid',
 'PopMG',
 'PopSA',
 'AddedCapGrid',
 'AddedCapSA',
 'AddedCapMG',
 'GridInv',
 'SAInv',
 'MGInv',
 'TotInv',
 'LCOEGrid',
 'LCOESA',
 'LCOEMGWind',
 'LCOEMGPV',
 'LCOEMGHydro',
 'LCOE',
 'Country']

df = df[cols]

df["RatioGridPop"] = 100*(df["PopGrid"]/df['Pop2030'])
df["RatioMGPop"] = 100*(df["PopMG"]/df['Pop2030'])
df["RatioSAPop"] = 100*(df["PopSA"]/df['Pop2030'])

df["RatioGridInv"] = 100*(df["GridInv"]/df['TotInv'])
df["RatioSAInv"] = 100*(df["SAInv"]/df['TotInv'])
df["RatioMGInv"] = 100*(df["MGInv"]/df['TotInv'])

df_shuffled = df.sample(frac=1)

In [ ]:
dfB = df_shuffled.loc[df['Country'] == 'bj']
dfN = df_shuffled.loc[df['Country'] == 'na']
dfM = df_shuffled.loc[df['Country'] == 'mw']
dfB['id'] = np.arange(len(dfB))
dfN['id'] = np.arange(len(dfN))
dfM['id'] = np.arange(len(dfM))

In [ ]:
problem_merged = read_param_file('params_merged.txt')
problem_bj =read_param_file('params_ben.txt')
problem_na = read_param_file('params_nam.txt')
problem_mw = read_param_file('params_mal.txt')

## Merged

In [ ]:
param_values=df_shuffled.iloc[:, : 16]
np.savetxt(r'model_input_merged.txt', param_values.values)
X_merged = np.loadtxt(r"model_input_merged.txt")

In [ ]:
out = df_shuffled["RatioMGPop"]
np.savetxt(r'model_output_merged.txt', out.values)
Y_merged = np.loadtxt(r"model_output_merged.txt")

In [ ]:
S_merged = delta.analyze(problem_merged, X_merged, Y_merged, num_resamples=10, conf_level=0.95, print_to_console=True)

## Benin

In [ ]:
param_values=dfB.iloc[:, : 14]
np.savetxt(r'model_input_ben.txt', param_values.values)
X_bj = np.loadtxt(r"model_input_ben.txt")

In [ ]:
out = dfB["RatioMGPop"]
np.savetxt(r'model_output_ben.txt', out.values)
Y_bj = np.loadtxt(r"model_output_ben.txt")

In [ ]:
Si_ben = delta.analyze(problem_bj, X_bj, Y_bj, num_resamples=10, conf_level=0.95, print_to_console=True)

## Malawi

In [ ]:
param_values=dfM.iloc[:, : 14]
np.savetxt(r'model_input_mal.txt', param_values.values)
X_mw = np.loadtxt(r"model_input_mal.txt")

In [ ]:
out = dfM["LCOE"]
np.savetxt(r'model_output_mal.txt', out.values)
Y_mw = np.loadtxt(r"model_output_mal.txt")

In [ ]:
Si_mal = delta.analyze(problem_mw, X_mw, Y_mw, num_resamples=10, conf_level=0.95, print_to_console=True)

## Namibia

In [ ]:
param_values=dfN.iloc[:, : 14]
np.savetxt(r'model_input_nam.txt', param_values.values)
X_na = np.loadtxt(r"model_input_nam.txt")

In [ ]:
out = dfN["LCOE"]
np.savetxt(r'model_output_nam.txt', out.values)
Y_na = np.loadtxt(r"model_output_nam.txt")

In [ ]:
Si_nam = delta.analyze(problem_na, X_na, Y_na, num_resamples=10, conf_level=0.95, print_to_console=True)

## Visualization

### Convergance

In [ ]:
df_shuffled = df.sample(frac=1)

In [ ]:
i = 40
delta_v = []
d_conf = []
S1_v = []
S1_conf = []

In [ ]:
while i <= 6240:
    df1x = df_shuffled[:i]
    
    param_values=df1x.iloc[:, : 16]
    np.savetxt(r'model_input_merged.txt', param_values.values)
    X = np.loadtxt(r"model_input_merged.txt")
    
    out = df1x["TotInv"]
    
    np.savetxt(r'model_output_merged.txt', out.values)
    Y = np.loadtxt(r"model_output_merged.txt")
    Si = delta.analyze(problem_merged, X, Y, num_resamples=10, conf_level=0.95, print_to_console=False)
    
    delta_v.append(Si['delta'].tolist())
    d_conf.append(Si['delta_conf'].tolist())
    S1_v.append(Si['S1'].tolist())
    S1_conf.append(Si['S1_conf'].tolist())
    i = i + 3000

In [ ]:
conf = pd.DataFrame(d_conf, columns = ['EDemand',
 'GridGenCost',
 'PVCost',
 'GridCapInvCost',
 'DiscountRate',
 'LVCost',
 'MVCost',
 'GridLosses',
 'NatPopDens',                           
 'Core',
 'Buffer',
 'Admin',
 'Method',
 'Res',
 'ElecRate',
 'Pop2030'])

delta_df = pd.DataFrame(delta_v, columns = ['EDemand',
 'GridGenCost',
 'PVCost',
 'GridCapInvCost',
 'DiscountRate',
 'LVCost',
 'MVCost',
 'GridLosses',
 'NatPopDens',                           
 'Core',
 'Buffer',
 'Admin',
 'Method',
 'Res',
 'ElecRate',
 'Pop2030'])


delta_df.to_csv("TotInv_delta.csv")
conf.to_csv("TotInv_conf.csv")

In [ ]:
merged_delta = pd.read_csv("TotInv_delta.csv", sep=None)
merged_delta.rename(columns={'Unnamed: 0':'Runs', 'EDemand':'Demand','Core':'minPts', 'Res':'Resolution', 'ElecRate': '2020ElecRate'}, inplace=True)

In [ ]:
merged_conf = pd.read_csv("TotInv_conf.csv", sep=None)
merged_conf.rename(columns={'Unnamed: 0':'Runs', 'EDemand':'Demand','Core':'minPts', 'Res':'Resolution', 'ElecRate': '2020ElecRate'}, inplace=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(15,20))
ax1.set_title('Delta value')
ax2.set_title('Confidence intervall')
sns.set(font_scale=1)

ax1.set_xlim([0, 6240])
ax1.set_ylim([0, 0.7])
ax2.set_xlim([0, 6240])
ax2.set_ylim([0, 0.2])

df_d = merged_delta.melt('Runs', var_name='cols',  value_name='vals')
df_d["Runs"] = df_d["Runs"]*200+40 
sns.pointplot(x="Runs", y="vals", hue='cols', data=df_d, kind='point', ax=ax1)
ax1.legend(bbox_to_anchor=(1.15, 1), borderaxespad=0)
ax1.set_ylabel('')

df_c = merged_conf.melt('Runs', var_name='cols',  value_name='vals')
df_c["Runs"] = df_c["Runs"]*200+40 
sns.pointplot(x="Runs", y="vals", hue='cols', data=df_c, kind='point', ax=ax2)
ax2.legend(bbox_to_anchor=(1.15, 1), borderaxespad=0)
ax2.set_ylabel('')

In [ ]:
fig.savefig("TotInv_delta.pdf", pad_inches=0.1,  bbox_inches = 'tight')
fig.savefig("TotInv_delta.png", pad_inches=0.1,  bbox_inches = 'tight')

In [ ]:
fig, axs = plt.subplots(3, figsize=(15,20))

colors = {0:'tab:cyan', 150:'tab:grey', 500:'tab:blue', 450:'tab:orange', 400:'tab:green', 350:'tab:red', 300:'tab:purple', 250:'tab:brown', 200:'tab:olive'}
colors = {100:'tab:cyan', 1000:'tab:grey'}

grouped = dfB.groupby('Res')
dfB["PopGrid2"] = dfB["PopSA"]/1000000
for key, group in grouped:
    group.plot(ax=axs[0], kind='scatter', x='id', y='PopGrid2', label=key, color=colors[key])
axs[0].get_legend().remove()
ax=axs[0].get_yaxis().get_major_formatter().set_scientific(False)
axs[0].set_xlabel("")
axs[0].set_ylabel("Benin", fontsize = 20)
axs[0].set_title("Effect of buffer on population connected to stand-alone PV", fontsize = 20)
axs[0].axes.get_xaxis().set_visible(False)

grouped = dfM.groupby('Res')
dfM["PopGrid2"] = dfM["PopSA"]/1000000
for key, group in grouped:
    group.plot(ax=axs[1], kind='scatter', x='id', y='PopGrid2', label=key, color=colors[key])
ax=axs[1].get_yaxis().get_major_formatter().set_scientific(False)
axs[1].set_xlabel("")
axs[1].set_ylabel("Malawi", fontsize = 20)
axs[1].legend(bbox_to_anchor=(1.01, 1), borderaxespad=0, title="Buffer (m)", fontsize = 15, title_fontsize=15)
axs[1].axes.get_xaxis().set_visible(False)

grouped = dfN.groupby('Res')
dfN["PopGrid2"] = dfN["PopSA"]/1000000
for key, group in grouped:
    group.plot(ax=axs[2], kind='scatter', x='id', y='PopGrid2', label=key, color=colors[key])
axs[2].get_legend().remove()
ax=axs[2].get_yaxis().get_major_formatter().set_scientific(False)
axs[2].set_xlabel("")
axs[2].set_ylabel("Namibia", fontsize = 20)
axs[2].axes.get_xaxis().set_visible(False)

fig.text(0.06, 0.5, 'Million people connected to stand-alone PV', ha='center', va='center', rotation='vertical', fontsize = 20)

plt.show()
fig.savefig("SAVSBuffer.png", dpi = 300, facecolor="white", edgecolor='none', bbox_inches='tight', pad_inches=0)
fig.savefig("SAVSBuffer.pdf", dpi = 300, facecolor="white", edgecolor='none', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(20,10))

ax1.set_ylabel('Delta value')

ax2.set_ylabel('Average LCoE (USD/kWh)')
ax2.set_xlabel('% of scenarios')
ax2.grid(True)


my_xticks = ['0%','25%', '50%', '75%', '100%']
x = np.array([0,520,1040,1560,2080])
plt.xticks(x, my_xticks)
ax2.set_xlim(0,2080)

Barr = dfB["LCOE"].to_numpy()
Narr = dfN["LCOE"].to_numpy()
Marr = dfM["LCOE"].to_numpy()

values_b, base_b = np.histogram(Barr, bins=2080)
values_m, base_m = np.histogram(Marr, bins=2080)
values_n, base_n = np.histogram(Narr, bins=2080)

cum_b = np.cumsum(values_b)
cum_n = np.cumsum(values_n)
cum_m = np.cumsum(values_m)

ax2.plot(cum_b, base_b[:-1], c='b')
ax2.plot(cum_m, base_m[:-1], c='g')
ax2.plot(cum_n, base_n[:-1], c='r')

colors = {'Benin':'b', 'Malawi':'g', 'Namibia':'r'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
ax2.legend(handles, labels, fontsize = 15, bbox_to_anchor=(0.98, 1))

my_xticks = ['Demand','GridGenCost','PVCost','GridCapInvCost','DiscountRate','LVCost','MVCost','GridLosses','Core','Buffer','Admin','Method','Resolution']

delta_ben = Si_ben["delta"].tolist()
delta_mal = Si_mal["delta"].tolist()
delta_nam = Si_nam["delta"].tolist()

data = [delta_ben, delta_mal, delta_nam]

X = np.arange(13)
ax1.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax1.bar(X + 0.25, data[1], color = 'g', width = 0.25, tick_label = my_xticks)
ax1.bar(X + 0.50, data[2], color = 'r', width = 0.25)

ax1.legend(handles, labels, fontsize = 15, bbox_to_anchor=(0.98, 1))

plt.show()

fig.savefig("LCOEVSDelta.png", dpi = 300, bbox_inches='tight', pad_inches=0)
fig.savefig("LCOEVSDelta.pdf", dpi = 300, bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(3, figsize=(15,20), sharex=True, squeeze=True)

dfM_I = dfM[["GridInv", "MGInv", "SAInv", "TotInv"]]/1000000000
dfM_IS = dfM_I.sort_values("TotInv") 

M_grid = dfM_IS['GridInv'].tolist()
M_MG = dfM_IS['MGInv'].tolist()
M_SA = dfM_IS['SAInv'].tolist()

dfB_I = dfB[["GridInv", "MGInv", "SAInv", "TotInv"]]/1000000000
dfB_IS = dfB_I.sort_values("TotInv") 

B_grid = dfB_IS['GridInv'].tolist()
B_MG = dfB_IS['MGInv'].tolist()
B_SA = dfB_IS['SAInv'].tolist()

dfN_I = dfN[["GridInv", "MGInv", "SAInv", "TotInv"]]/1000000000
dfN_IS = dfN_I.sort_values("TotInv") 

N_grid = dfN_IS['GridInv'].tolist()
N_MG = dfN_IS['MGInv'].tolist()
N_SA = dfN_IS['SAInv'].tolist()


population_by_continent = {
    'Mini-grid': B_MG,
    'Stand-alone': B_SA,
    'Grid': B_grid
}
axs[0].stackplot(range(1,2081), population_by_continent.values(),
             labels=population_by_continent.keys(),edgecolor="#ffffff00")

axs[0].get_yaxis().get_major_formatter().set_scientific(False)
axs[0].set_ylabel("Benin", fontsize = 20)
axs[0].set_title("Stacked area chart of investment cost per category", fontsize = 20)

population_by_continent = {
    'Mini-grid': M_MG,
    'Stand-alone': M_SA,
    'Grid': M_grid
}
axs[1].stackplot(range(1,2081), population_by_continent.values(),
             labels=population_by_continent.keys(),edgecolor="#ffffff00")

ax=axs[1].get_yaxis().get_major_formatter().set_scientific(False)
axs[1].set_ylabel("Malawi", fontsize = 20)
axs[1].set_xlim(0,2080)

axs[1].legend(bbox_to_anchor=(1.17, 1), borderaxespad=0, title="Category", fontsize = 15, title_fontsize=15)

population_by_continent = {
    'Mini-grid': N_MG,
    'Stand-alone': N_SA,
    'Grid': N_grid
}
axs[2].stackplot(range(1,2081), population_by_continent.values(),
             labels=population_by_continent.keys(),edgecolor="#ffffff00")

my_xticks = ['0%','25%', '50%', '75%', '100%']
x = np.array([0,520,1040,1560,2080])
plt.xticks(x, my_xticks)
axs[2].set_xlim(0,2080)

ax=axs[2].get_yaxis().get_major_formatter().set_scientific(False)
axs[2].set_xlabel("% of scenarios")
axs[2].set_ylabel("Namibia", fontsize = 20)


fig.text(0.06, 0.5, 'Investment cost (billion USD)', ha='center', va='center', rotation='vertical', fontsize = 20)

plt.show()

fig.savefig("InvCost.png", dpi = 300, bbox_inches='tight', pad_inches=0)
fig.savefig("InvCost.pdf", dpi = 300, bbox_inches='tight', pad_inches=0)